<a href="https://colab.research.google.com/github/Vergosss/Thesis/blob/main/BERTOnLogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
import torch
from sklearn.model_selection import train_test_split
##
from google.colab import drive
drive.mount('/content/drive')
##
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#event vectors
event_traces = pd.read_csv('/content/drive/MyDrive/Data/Event_traces.csv',usecols=['BlockId','Label','Features'])
event_traces['Label'] = event_traces['Label'].map({'Success':1,'Fail':0})
#event template- message
log_templates = pd.read_csv('/content/drive/MyDrive/Data/HDFS.log_templates.csv')
#Printing-Displaying
display(HTML((event_traces.sample(5)).to_html()))
#
display(HTML((log_templates.sample(5)).to_html()))
##Dictionary ton EventIds-Events
event_dictionary = dict(zip(log_templates['EventId'],log_templates['EventTemplate']))

print(event_dictionary)
print(event_traces['Features'].sample(1))
event_traces.info()
#input('WAIT')
##
def features_to_strings(entry):
  return [event_dictionary.get(eventID) for eventID in entry['Features'].replace('[','').replace(']','').split(',')]

event_traces['Features'] = event_traces.apply(features_to_strings,axis=1)
display(HTML((event_traces.sample(1)).to_html()))
input('WAIT')
##
'''
#
logs_csv = pd.read_csv('/content/drive/MyDrive/Data/BGL.csv')
#print('First 5 entries: ',logs_csv.head())
display(HTML((logs_csv.sample(5)).to_html()))
##
#print('Last 5 entries: ',logs_csv.tail())
labels = logs_csv['Label'].unique()
print(labels)
#
logs_csv['Label'] = logs_csv['Label'].map({'-':0})
logs_csv['Label'] = logs_csv['Label'].fillna(1).astype(int)
#
display(HTML((logs_csv.sample(5)).to_html()))
###
print(logs_csv['Label'].value_counts())


'''
##

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # tokenizer
#orisma ena line tou dataset opote epilego to column me to keimeno pou thelo na tokenaro
def tokenize_logs(entry):
  return tokenizer(entry['Content'],padding='max_length',truncation=True)
#
'''
logs_csv['Content'] = logs_csv.apply(tokenize_logs,axis=1)
#

#
display(HTML((logs_csv.sample(5)).to_html()))
##
drop_columns = ['LineId','Timestamp','Date','Node','Time','NodeRepeat','Type','Component','Level','EventId','EventTemplate']
clean_logs_csv = logs_csv.drop(columns=drop_columns)
display(HTML((clean_logs_csv.sample(5)).to_html()))
'''
##
#X_train,X_test,y_train,y_test = train_test_split(clean_logs_csv['Content'],clean_logs_csv['Label'],test_size=0.2,random_state=42)
#print(X_train[0])
#########Fine tuning meso ton parakato parametron##########

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2).to(device)#modelo me ena extra layer gia to classification
###
training_arguments = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
     per_device_train_batch_size=16,
    per_device_eval_batch_size=64

)

#########
trainer = Trainer(

    model=model,#modelo pou tha kanei to classification,

    args=training_args,
    tokenizer=tokenizer#o tokenizer,

    #data_collator=data_collator,

)
### Train the model
#trainer.train()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cpu


,BlockId,Label,Features
410983,blk_-7397595242157932421,1,"[E5,E5,E5,E22,E11,E9,E11,E9,E26,E26,E11,E9,E26,E23,E23,E23,E21,E21,E21]"
95726,blk_962223603793551040,1,"[E22,E5,E5,E5,E26,E26,E11,E9,E11,E9,E11,E9,E26,E3,E4,E3,E4,E3,E3,E4,E3,E3,E23,E23,E23,E21,E21,E21]"
377626,blk_5465670254893690827,1,"[E5,E5,E5,E22,E11,E9,E11,E9,E11,E9,E26,E26,E26,E23,E23,E23,E21,E21,E21]"
50210,blk_1910008936831073547,1,"[E22,E5,E5,E5,E26,E26,E26,E11,E9,E11,E9,E11,E9,E23,E23,E23,E21,E21,E21]"
349507,blk_-6861825879037213300,1,"[E5,E22,E5,E5,E11,E9,E11,E9,E11,E9,E26,E26,E26]"


,EventId,EventTemplate
28,E29,[*]PendingReplicationMonitor timed out block[*]
23,E24,[*]BLOCK* Removing block[*]from neededReplications as it does not belong to any file[*]
14,E15,[*]Changing block file offset of block[*]from[*]to[*]meta file offset to[*]
0,E1,[*]Adding an already existing block[*]
3,E4,[*]Got exception while serving[*]to[*]


{'E1': '[*]Adding an already existing block[*]', 'E2': '[*]Verification succeeded for[*]', 'E3': '[*]Served block[*]to[*]', 'E4': '[*]Got exception while serving[*]to[*]', 'E5': '[*]Receiving block[*]src:[*]dest:[*]', 'E6': '[*]Received block[*]src:[*]dest:[*]of size[*]', 'E7': '[*]writeBlock[*]received exception[*]', 'E8': '[*]PacketResponder[*]for block[*]Interrupted[*]', 'E9': '[*]Received block[*]of size[*]from[*]', 'E10': '[*]PacketResponder[*]Exception[*]', 'E11': '[*]PacketResponder[*]for block[*]terminating[*]', 'E12': '[*]:Exception writing block[*]to mirror[*]', 'E13': '[*]Receiving empty packet for block[*]', 'E14': '[*]Exception in receiveBlock for block[*]', 'E15': '[*]Changing block file offset of block[*]from[*]to[*]meta file offset to[*]', 'E16': '[*]:Transmitted block[*]to[*]', 'E17': '[*]:Failed to transfer[*]to[*]got[*]', 'E18': '[*]Starting thread to transfer block[*]to[*]', 'E19': '[*]Reopen Block[*]', 'E20': '[*]Unexpected error trying to delete block[*]BlockInfo 

,BlockId,Label,Features
553786,blk_-4650489431364790352,1,"[[*]Receiving block[*]src:[*]dest:[*], [*]Receiving block[*]src:[*]dest:[*], [*]Receiving block[*]src:[*]dest:[*], [*]BLOCK* NameSystem[*]allocateBlock:[*], [*]PacketResponder[*]for block[*]terminating[*], [*]Received block[*]of size[*]from[*], [*]PacketResponder[*]for block[*]terminating[*], [*]Received block[*]of size[*]from[*], [*]PacketResponder[*]for block[*]terminating[*], [*]Received block[*]of size[*]from[*], [*]BLOCK* NameSystem[*]addStoredBlock: blockMap updated:[*]is added to[*]size[*], [*]BLOCK* NameSystem[*]addStoredBlock: blockMap updated:[*]is added to[*]size[*], [*]BLOCK* NameSystem[*]addStoredBlock: blockMap updated:[*]is added to[*]size[*], [*]BLOCK* NameSystem[*]delete:[*]is added to invalidSet of[*], [*]BLOCK* NameSystem[*]delete:[*]is added to invalidSet of[*], [*]BLOCK* NameSystem[*]delete:[*]is added to invalidSet of[*], [*]Deleting block[*]file[*], [*]Deleting block[*]file[*], [*]Deleting block[*]file[*]]"


KeyboardInterrupt: Interrupted by user